### Import and config

In [1]:
# Imports
import os
import time
import random
import json
import logging

from dotenv import load_dotenv
from tqdm import tqdm

import pandas as pd
import requests

from sqlalchemy import create_engine, text, inspect, Text, Integer, BigInteger, Float, Boolean
from sqlalchemy.dialects.postgresql import JSONB

# Configuration
load_dotenv()

# API urls
AUTH_URL = os.getenv('AUTH_URL')
ACTIVITIES_URL = os.getenv('ACTIVITIES_URL')
ACTIVITY_DETAIL_URL_TMPL = os.getenv('ACTIVITY_DETAIL_URL_TMPL')

# API keys
CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')
REFRESH_TOKEN = os.getenv('REFRESH_TOKEN')

# DB
DB_URI = os.getenv('DB_URI')

# Bronze tables
TARGET_B_SCHEMA = os.getenv('TARGET_B_SCHEMA')
ACTIVITIES_B_TABLE = os.getenv('ACTIVITIES_B_TABLE')
DETAILS_B_TABLE = os.getenv('DETAILS_B_TABLE')

# Pagination
PER_PAGE = int(os.getenv('PER_PAGE'))
MAX_PAGES = int(os.getenv('MAX_PAGES'))

# Timeouts and retries
REQUEST_TIMEOUT = int(os.getenv('REQUEST_TIMEOUT'))
MAX_RETRIES = int(os.getenv('MAX_RETRIES'))
BASE_SLEEP = float(os.getenv('BASE_SLEEP'))

# Other
LOG_LEVEL = os.getenv('LOG_LEVEL')

logging.basicConfig(
    level=getattr(logging, LOG_LEVEL.upper(), logging.INFO),
    format="%(asctime)s | %(levelname)s | %(message)s"
)

### API keys validation

In [2]:
REQUIRED_API_ENV = ['CLIENT_ID', 'CLIENT_SECRET', 'REFRESH_TOKEN']
missing_api_env = [env for env in REQUIRED_API_ENV if not os.getenv(env)]
if missing_api_env:
  raise RuntimeError(f"Missing env variables: {', '.join(missing_api_env)}.")

### DB names validation

In [ ]:
REQUIRED_DB_ENV = ['DB_URI', 'TARGET_B_SCHEMA', 'ACTIVITIES_B_TABLE', 'DETAILS_B_TABLE']
missing_db_env = [env for env in REQUIRED_DB_ENV if not os.getenv(env)]
if missing_db_env:
  raise RuntimeError(f"Missing env variables: {', '.join(missing_db_env)}.")

### Connecting with PostgreSQL

In [3]:
engine = create_engine(
  DB_URI, 
  pool_pre_ping=True, 
  pool_size=5, 
  max_overflow=10
)
with engine.connect() as conn:
  conn.execute(text(f"CREATE SCHEMA IF NOT EXISTS {TARGET_B_SCHEMA};"))
logging.info(f"Connection established and {TARGET_B_SCHEMA} schema exists.")

2025-09-16 19:12:43,980 | INFO | Connection established and bronze schema exists.


### Token authorization

In [16]:
def get_access_token(auth_url: str, client_id: str, client_secret: str, refresh_token: str, timeout: int = 30) -> str:
  """
  Request a new access token from an OAuth2 authentication endpoint.

  The function uses a refresh token to obtain a short-lived access token. 
  If the response does not contain an ``access_token``, a RuntimeError is raised.

  Parameters
  ----------
  auth_url : str
      URL of the OAuth2 token endpoint.
  client_id : str
      OAuth2 client identifier.
  client_secret : str
      OAuth2 client secret.
  refresh_token : str
      Refresh token used to request a new access token.
  timeout : int, default=30
      Timeout in seconds for the HTTP request.

  Returns
  -------
  str
      The access token string retrieved from the authentication server.

  Raises
  ------
  RuntimeError
      If the response does not include an ``access_token``.
  requests.exceptions.RequestException
      If the HTTP request fails (e.g., network error, timeout).

  Notes
  -----
  Logs an informational message when the token is successfully retrieved.
  """

  payload = {
    'client_id': client_id,
    'client_secret': client_secret,
    'refresh_token': refresh_token,
    'grant_type': 'refresh_token',
  }
  res = requests.post(auth_url, data=payload, timeout=timeout)
  data = res.json()
  token = data.get('access_token')
  if not token:
    raise RuntimeError(f"No access token in response: {data}")
  logging.info('Access token retrived.')
  return token

access_token = get_access_token(AUTH_URL, CLIENT_ID, CLIENT_SECRET, REFRESH_TOKEN)

2025-09-17 12:51:19,533 | INFO | Access token retrived.


### HTTP session

In [5]:
session = requests.Session()
session.headers.update({"Authorization": f"Bearer {access_token}"})

def get_json_with_retry(url: str, params=None, max_retries: int = MAX_RETRIES, timeout: int = REQUEST_TIMEOUT, base_sleep: float = BASE_SLEEP):
  """
  Send a GET request with automatic retries and return the parsed JSON response.

  The function retries on:
    * HTTP 429 (rate limiting) — respects the ``Retry-After`` header if present,
      otherwise waits an increasing backoff time.
    * HTTP 5xx errors — retries with exponential backoff and jitter.
    * Network/connection errors — retries with exponential backoff and jitter.

  For each attempt, the backoff time increases by ``base_sleep * attempt`` seconds
  plus a small random jitter. On the final attempt, any error is raised.

  Parameters
  ----------
  url : str
      Endpoint URL to send the GET request to.
  params : dict, optional
      Query string parameters to include in the request.
  max_retries : int, default=MAX_RETRIES
      Maximum number of retry attempts before failing.
  timeout : int, default=REQUEST_TIMEOUT
      Timeout in seconds for each HTTP request.
  base_sleep : float, default=BASE_SLEEP
      Base number of seconds used for exponential backoff between retries.

  Returns
  -------
  dict
      Parsed JSON response from the server.

  Raises
  ------
  requests.exceptions.RequestException
      If the request fails after all retry attempts.
  requests.exceptions.HTTPError
      If the server returns a 4xx/5xx response on the last attempt.
  json.JSONDecodeError
      If the response cannot be parsed as JSON.

  Notes
  -----
  * HTTP 429 triggers a wait using ``Retry-After`` if available, otherwise a fallback.
  * Logs warnings before each retry and errors if JSON parsing fails.
  * Uses a global ``requests.Session`` (`session`) for connection pooling.
  """

  for attempt in range(1, max_retries + 1):
    try:
      resp = session.get(url, params=params, timeout=timeout)
      
      if resp.status_code == 429:
        retry_after = resp.headers.get('Retry-After')
        if retry_after and retry_after.isdigit():
          sleep_for = int(retry_after)
        else:
          sleep_for = max(base_sleep * attempt, 15)
        logging.warning(f"HTTP 429 - wait {sleep_for}s (attempt {attempt}/{max_retries})")
        time.sleep(sleep_for)
        continue

      if 500 <= resp.status_code < 600:
        if attempt == max_retries:
          resp.raise_for_status()
        sleep_for = base_sleep * attempt + random.uniform(0, 1.0)
        logging.warning(f"HTTP {resp.status_code} — retry in {sleep_for:.1f}s (attempt {attempt}/{max_retries})")
        time.sleep(sleep_for)
        continue

      resp.raise_for_status()

      try:
        return resp.json()
      except json.JSONDecodeError:
        logging.error('JSON parsing error')
        raise
    
    except requests.exceptions.RequestException as e:
      if attempt == max_retries:
        logging.exception('Request error (last attempt)')
        raise
      sleep_for = base_sleep * attempt + random.uniform(0, 1.0)
      logging.warning(f"{e} — retry in {sleep_for:.1f}s (attempt {attempt}/{max_retries})")
      time.sleep(sleep_for)

### Getting activities list

In [6]:
def fetch_all_activities(activites_url: str, per_page: int = PER_PAGE, max_pages: int = MAX_PAGES):
  """
  Fetch all activity records from a paginated API endpoint.

  The function iterates through API pages until either:
    * the maximum number of pages is reached (``max_pages``), or
    * the API returns an empty list (end of results).

  Each page is retrieved using ``get_json_with_retry`` to ensure resilience
  against transient errors (rate limits, timeouts, 5xx responses).

  Parameters
  ----------
  activites_url : str
      The base URL of the activities endpoint (must support ``per_page`` and ``page`` query params).
  per_page : int, default=PER_PAGE
      Number of activity records to request per page.
  max_pages : int, default=MAX_PAGES
      Maximum number of pages to fetch before stopping.

  Returns
  -------
  list of dict
      Combined list of activity objects returned by the API.

  Raises
  ------
  RuntimeError
      If the API response is not a list (unexpected schema).
  requests.exceptions.RequestException
      If the underlying HTTP requests fail after retries.

  Notes
  -----
  * Logs the number of records downloaded per page and the running total.
  * Stops early if the API returns an empty list before reaching ``max_pages``.
  * The total number of records is ``per_page * n_pages`` at most.
  """

  all_items = []
  page = 1
  while page <= max_pages:
    params = {'per_page': per_page, 'page': page}
    data = get_json_with_retry(activites_url, params=params)

    if not isinstance(data, list):
      raise RuntimeError(f"Unexpected response type for page {page}: {type(data)} — expected list")
    
    if not data:
      break
    
    all_items.extend(data)
    logging.info(f"Page {page}: downloaded {len(data)} records (total: {len(all_items)})")
    page += 1
  return all_items

activities_raw = fetch_all_activities(ACTIVITIES_URL, per_page=PER_PAGE, max_pages=MAX_PAGES)
logging.info(f"Total activities downloaded: {len(activities_raw)}")
activities_df = pd.json_normalize(activities_raw, sep='_')
if activities_df.empty:
    raise RuntimeError('No activities to save.')
activities_df.head()

2025-09-16 19:12:49,275 | INFO | Page 1: downloaded 200 records (total: 200)
2025-09-16 19:12:53,228 | INFO | Page 2: downloaded 200 records (total: 400)
2025-09-16 19:12:57,582 | INFO | Page 3: downloaded 200 records (total: 600)
2025-09-16 19:13:00,824 | INFO | Page 4: downloaded 200 records (total: 800)
2025-09-16 19:13:04,634 | INFO | Page 5: downloaded 200 records (total: 1000)
2025-09-16 19:13:05,291 | INFO | Page 6: downloaded 93 records (total: 1093)
2025-09-16 19:13:05,483 | INFO | Total activities downloaded: 1093


,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,id,start_date,...,map_summary_polyline,map_resource_state,workout_type,average_cadence,average_watts,max_watts,weighted_average_watts,device_watts,kilojoules,average_temp
0,2,Afternoon Weight Training,0.0,3825,3825,0.0,Workout,WeightTraining,15831049874,2025-09-16T13:01:07Z,...,,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Tempo 2km🥵,9521.9,3241,3241,13.0,Run,Run,15820198827,2025-09-15T14:23:21Z,...,{_|vH{pogBx@`B\x@HJRDN?|@Wj@GJ?\Rt@vABRBhAD`AT...,2,3.0,83.9,353.5,493.0,369.0,True,1144.0,NaN
2,2,15km Long Run☔️,15059.0,5461,5488,31.0,Run,Run,15805849875,2025-09-14T07:59:25Z,...,w_|vHqpogBrAnCNj@@l@Gb@o@hAa@h@QXuCpDyDbEsApAk...,2,2.0,86.0,335.5,455.0,334.0,True,1833.0,NaN
3,2,Tempo 4km😮‍💨,7531.2,2400,2400,11.0,Run,Run,15798063578,2025-09-13T13:59:16Z,...,us{vHm~ngBJ`@P`@Nt@\b@j@`BXj@`@bAf@|ATz@j@xDJ`...,2,3.0,83.9,364.2,509.0,377.0,True,873.1,NaN
4,2,6km Easy Run😌,6062.9,2140,2140,17.0,Run,Run,15786538213,2025-09-12T13:40:46Z,...,kh|vHguogBJ\JNHFRBN?fBc@\FZ`@JXdAzBp@xAHFRBNAh...,2,NaN,83.1,351.5,445.0,350.0,True,752.4,NaN


### Set `bronze.activities` types map

In [7]:
activities_dtype_map = {
    "resource_state": Integer,
    "name": Text,
    "distance": Float,
    "moving_time": Integer,
    "elapsed_time": Integer,
    "total_elevation_gain": Float,
    "type": Text,
    "sport_type": Text,
    "workout_type": Float,
    "id": BigInteger,
    "start_date": Text,
    "start_date_local": Text,
    "timezone": Text,
    "utc_offset": Float,
    "location_city": Text,
    "location_state": Text,
    "location_country": Text,
    "achievement_count": Integer,
    "kudos_count": Integer,
    "comment_count": Integer,
    "athlete_count": Integer,
    "photo_count": Integer,
    "trainer": Boolean,
    "commute": Boolean,
    "manual": Boolean,
    "private": Boolean,
    "visibility": Text,
    "flagged": Boolean,
    "gear_id": Text,
    "start_latlng": JSONB,
    "end_latlng": JSONB,
    "average_speed": Float,
    "max_speed": Float,
    "average_cadence": Float,
    "average_watts": Float,
    "max_watts": Float,
    "weighted_average_watts": Float,
    "device_watts": Boolean,
    "kilojoules": Float,
    "has_heartrate": Boolean,
    "average_heartrate": Float,
    "max_heartrate": Float,
    "heartrate_opt_out": Boolean,
    "display_hide_heartrate_option": Boolean,
    "elev_high": Float,
    "elev_low": Float,
    "upload_id": BigInteger,
    "upload_id_str": Text,
    "external_id": Text,
    "from_accepted_tag": Boolean,
    "pr_count": Integer,
    "total_photo_count": Integer,
    "has_kudoed": Boolean,
    "suffer_score": Float,
    "athlete_id": BigInteger,
    "athlete_resource_state": Integer,
    "map_id": Text,
    "map_summary_polyline": Text,
    "map_resource_state": Integer,
    "average_temp": Float,
}

### Save `bronze.activities` to database

In [8]:
table_full_name = f"{TARGET_B_SCHEMA}.{ACTIVITIES_B_TABLE}"
logging.warning(f"Whole table {table_full_name} will be overwritten.")
activities_df.to_sql(
  ACTIVITIES_B_TABLE, 
  engine, 
  schema=TARGET_B_SCHEMA, 
  if_exists="replace", 
  index=False, 
  method="multi", 
  chunksize=1000, 
  dtype=activities_dtype_map
)

2025-09-16 19:13:05,531 | WARNING | Whole table bronze.activities will be overwritten.


-2

### Identify missing details — only `id` that are not present in `bronze.activities_details`

In [9]:
with engine.begin() as conn:
  conn.execute(text(f"""
    CREATE TABLE IF NOT EXISTS {TARGET_B_SCHEMA}.{ACTIVITIES_B_TABLE} (id BIGINT PRIMARY KEY)
"""))
  existing_ids = pd.read_sql(text(f"SELECT id FROM {TARGET_B_SCHEMA}.{DETAILS_B_TABLE}"), conn)
  existing_ids_set = set(existing_ids['id'].astype('Int64').dropna().to_list())

all_ids_set = set(activities_df['id'].astype('Int64').dropna().to_list())
missing_ids = sorted(all_ids_set - existing_ids_set)
logging.info(f"Missing details of {len(missing_ids)} IDs")
pd.DataFrame({'id': missing_ids}).head()

2025-09-16 19:13:06,402 | INFO | Missing details of 2 IDs


,id
0,15820198827
1,15831049874


### Download activities details

In [10]:
def fetch_activity_details(activity_id: int):
  """
  Fetch detailed information for a single activity.

  Builds the activity detail endpoint URL using the given activity ID
  and retrieves the JSON payload with retry logic.

  Parameters
  ----------
  activity_id : int
      Unique identifier of the activity.

  Returns
  -------
  dict
      JSON object containing the activity details as returned by the API.

  Raises
  ------
  requests.exceptions.RequestException
      If the request fails after retries (e.g., network error, rate limit, 5xx).
  requests.exceptions.HTTPError
      If the server returns an error response on the last attempt.
  json.JSONDecodeError
      If the response cannot be parsed as JSON.

  Notes
  -----
  * Uses the global template ``ACTIVITY_DETAIL_URL_TMPL`` to construct the URL.
  * Under the hood calls ``get_json_with_retry`` for resiliency.
  """
  url = ACTIVITY_DETAIL_URL_TMPL.format(id=activity_id)
  return get_json_with_retry(url, params=None)

details_records = []
for i, act_id in tqdm(enumerate(missing_ids, start=1), total=len(missing_ids)):
  try:
    resp = fetch_activity_details(act_id)
    
    if not isinstance(resp, dict):
      logging.warning(f"id={act_id}: unexpected response type ({type(resp)}), skip")
      continue
    details_records.append(resp)
    
  except Exception as e:
    logging.error(f"Error downloading details for id={act_id}: {e}")
  
  time.sleep(random.randint(7, 9))

logging.info(f"Details downloaded: {len(details_records)} / {len(missing_ids)}")
details_df_new = pd.json_normalize(details_records, sep='_')

100%|██████████| 2/2 [00:19<00:00,  9.50s/it]
2025-09-16 19:13:25,435 | INFO | Details downloaded: 2 / 2


In [11]:
if not details_df_new.empty:
  details_df_new = details_df_new.drop(['message', 'errors'], axis=1, errors='ignore')
  details_df_new = details_df_new.dropna(how='all')
  
details_df_new.head()

,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,id,...,similar_activities_max_average_speed,similar_activities_pr_rank,similar_activities_frequency_milestone,similar_activities_trend_speeds,similar_activities_trend_current_activity_index,similar_activities_trend_min_speed,similar_activities_trend_mid_speed,similar_activities_trend_max_speed,similar_activities_trend_direction,similar_activities_resource_state
0,3,Tempo 2km🥵,9521.9,3241,3241,13.0,Run,Run,3.0,15820198827,...,2.944055,3.0,NaN,"[2.8269134384090804, 2.813875939948858, 2.8199...",4.0,2.410429,2.650651,2.944055,1.0,2.0
1,3,Afternoon Weight Training,0.0,3825,3825,0.0,Workout,WeightTraining,NaN,15831049874,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Set `bronze.activities_details` types map

In [12]:
activities_details_dtype_map = {
    "resource_state": Integer,
    "name": Text,
    "distance": Float,
    "moving_time": Integer,
    "elapsed_time": Integer,
    "total_elevation_gain": Float,
    "type": Text,
    "sport_type": Text,
    "workout_type": Float,
    "id": BigInteger,
    "start_date": Text,
    "start_date_local": Text,
    "timezone": Text,
    "utc_offset": Float,
    "location_city": Text,
    "location_state": Text,
    "location_country": Text,
    "achievement_count": Integer,
    "kudos_count": Integer,
    "comment_count": Integer,
    "athlete_count": Integer,
    "photo_count": Integer,
    "trainer": Boolean,
    "commute": Boolean,
    "manual": Boolean,
    "private": Boolean,
    "visibility": Text,
    "flagged": Boolean,
    "gear_id": Text,
    "start_latlng": JSONB,
    "end_latlng": JSONB,
    "average_speed": Float,
    "max_speed": Float,
    "average_cadence": Float,
    "average_watts": Float,
    "max_watts": Float,
    "weighted_average_watts": Float,
    "device_watts": Boolean,
    "kilojoules": Float,
    "has_heartrate": Boolean,
    "average_heartrate": Float,
    "max_heartrate": Float,
    "heartrate_opt_out": Boolean,
    "display_hide_heartrate_option": Boolean,
    "elev_high": Float,
    "elev_low": Float,
    "upload_id": BigInteger,
    "upload_id_str": Text,
    "external_id": Text,
    "from_accepted_tag": Boolean,
    "pr_count": Integer,
    "total_photo_count": Integer,
    "has_kudoed": Boolean,
    "suffer_score": Float,
    "description": Text,
    "calories": Float,
    "perceived_exertion": Text,
    "prefer_perceived_exertion": Text,
    "segment_efforts": JSONB,
    "splits_metric": JSONB,
    "splits_standard": JSONB,
    "laps": JSONB,
    "best_efforts": JSONB,
    "stats_visibility": JSONB,
    "hide_from_home": Boolean,
    "device_name": Text,
    "embed_token": Text,
    "available_zones": JSONB,
    "athlete_id": BigInteger,
    "athlete_resource_state": Integer,
    "map_id": Text,
    "map_polyline": Text,
    "map_resource_state": Integer,
    "map_summary_polyline": Text,
    "gear_primary": Boolean,
    "gear_name": Text,
    "gear_nickname": Text,
    "gear_resource_state": Float,
    "gear_retired": Boolean,
    "gear_distance": Float,
    "gear_converted_distance": Float,
    "photos_primary": JSONB,
    "photos_count": Integer,
    "similar_activities_effort_count": Float,
    "similar_activities_average_speed": Float,
    "similar_activities_min_average_speed": Float,
    "similar_activities_mid_average_speed": Float,
    "similar_activities_max_average_speed": Float,
    "similar_activities_pr_rank": Float,
    "similar_activities_frequency_milestone": Float,
    "similar_activities_trend_speeds": JSONB,
    "similar_activities_trend_current_activity_index": Float,
    "similar_activities_trend_min_speed": Float,
    "similar_activities_trend_mid_speed": Float,
    "similar_activities_trend_max_speed": Float,
    "similar_activities_trend_direction": Float,
    "similar_activities_resource_state": Float,
    "average_temp": Float,
    "photos_primary_unique_id": Text,
    "photos_primary_urls_600": Text,
    "photos_primary_urls_100": Text,
    "photos_primary_source": Integer,
    "photos_primary_media_type": Integer,
    "photos_use_primary_photo": Boolean,
    "private_note": Text
}

### Save `bronze.activities_details` to database

In [13]:
if details_df_new.empty:
  logging.info('No new details to be saved.')
else:
  insp = inspect(engine)

  # 1) Check if table exists in PostgreSQL
  if not insp.has_table(table_name=DETAILS_B_TABLE, schema=TARGET_B_SCHEMA):
    details_df_new[0].to_sql(
      DETAILS_B_TABLE, 
      engine, 
      schema=TARGET_B_SCHEMA,
      if_exists="append", 
      index=False, 
      dtype=activities_details_dtype_map
    )

    # 2) Check if primary key exists
  insp = inspect(engine)
  pk = insp.get_pk_constraint(table_name=DETAILS_B_TABLE, schema=TARGET_B_SCHEMA)
  if not pk.get("constrained_columns"):
    with engine.begin() as conn:
      conn.execute(text(f'''
        ALTER TABLE {TARGET_B_SCHEMA}.{DETAILS_B_TABLE}
        ALTER COLUMN "id" SET NOT NULL;
        ALTER TABLE {TARGET_B_SCHEMA}.{DETAILS_B_TABLE}
        ADD PRIMARY KEY ("id");
      '''
      ))
    
    # 3) Staging
  with engine.begin() as conn:

    #Temp table
    conn.execute(text(f'''
        CREATE TEMP TABLE details_stg
        AS TABLE {TARGET_B_SCHEMA}.{DETAILS_B_TABLE} WITH NO DATA
      '''
      ))
    
    # Load data to temp table
    details_df_new.to_sql(
      "details_stg", 
      conn, 
      if_exists="append",
      index=False, 
      method="multi", 
      chunksize=5000, 
      dtype=activities_details_dtype_map
      )
    
    # Prepare upsert
    stg_cols = details_df_new.columns.to_list()
    
    if not 'id' in stg_cols:
      raise RuntimeError('Column "id" is required in details_df_new to perform UPSERT.')
    
    cols_csv = ', '.join(f'"{c}"' for c in stg_cols)
    set_csv  = ', '.join(f'"{c}" = EXCLUDED."{c}"' for c in stg_cols if c != 'id')

    upsert_sql = f'''
        INSERT INTO {TARGET_B_SCHEMA}.{DETAILS_B_TABLE} ({cols_csv})
        SELECT {cols_csv} FROM details_stg
        ON CONFLICT ("id") DO UPDATE
        SET {set_csv};
      '''
    conn.execute(text(upsert_sql))

    # Clean up
    conn.execute(text('DROP TABLE IF EXISTS details_stg;'))
  
    logging.info('Activities details saved to PostgreSQL.')

2025-09-16 19:13:25,508 | INFO | Activities details saved to PostgreSQL.


### Sanity check

In [14]:
with engine.begin() as conn:
    total_acts = pd.read_sql(text(f"SELECT COUNT(*) AS n FROM {TARGET_B_SCHEMA}.{ACTIVITIES_B_TABLE}"), conn)
    total_det = pd.read_sql(text(f"SELECT COUNT(*) AS n FROM {TARGET_B_SCHEMA}.{DETAILS_B_TABLE}"), conn)

logging.info(f"Total number of records in {TARGET_B_SCHEMA}.{ACTIVITIES_B_TABLE}: {int(total_acts['n'][0])}")
logging.info(f"Total number of records in {TARGET_B_SCHEMA}.{DETAILS_B_TABLE}: {int(total_det['n'][0])}")

2025-09-16 19:13:25,526 | INFO | Total number of records in bronze.activities: 1093
2025-09-16 19:13:25,527 | INFO | Total number of records in bronze.activities_details: 1093


In [34]:
session = requests.Session()
session.headers.update({"Authorization": f"Bearer {access_token}"})
resp = fetch_activity_details(15729456618)

In [35]:
df = pd.json_normalize(resp)

In [36]:
df['visibility']

0    everyone
Name: visibility, dtype: object

In [37]:
pd.set_option('display.max_columns', None)
df

,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,id,start_date,start_date_local,timezone,utc_offset,location_city,location_state,location_country,achievement_count,kudos_count,comment_count,athlete_count,photo_count,trainer,commute,manual,private,visibility,flagged,gear_id,start_latlng,end_latlng,average_speed,max_speed,average_temp,average_watts,device_watts,kilojoules,has_heartrate,average_heartrate,max_heartrate,heartrate_opt_out,display_hide_heartrate_option,elev_high,elev_low,upload_id,upload_id_str,external_id,from_accepted_tag,pr_count,total_photo_count,has_kudoed,suffer_score,description,calories,perceived_exertion,prefer_perceived_exertion,segment_efforts,splits_metric,splits_standard,laps,stats_visibility,hide_from_home,device_name,embed_token,private_note,available_zones,athlete.id,athlete.resource_state,map.id,map.polyline,map.resource_state,map.summary_polyline,gear.id,gear.primary,gear.name,gear.nickname,gear.resource_state,gear.retired,gear.distance,gear.converted_distance,photos.primary,photos.count
0,3,Lunch Ride,79588.5,11082,14430,202.0,Ride,Ride,10,15729456618,2025-09-07T09:45:26Z,2025-09-07T11:45:26Z,(GMT+01:00) Europe/Warsaw,7200.0,None,None,None,44,14,0,1,0,False,False,False,False,everyone,False,b12572672,"[51.108316, 17.123345]","[51.107901, 17.123794]",7.182,11.18,23,187.7,False,2080.5,True,129.0,148.0,False,True,158.4,115.4,16801924720,16801924720,garmin_ping_477733386080,False,20,0,False,53.0,Nogi nie współpracowały po wczorajszym longu🪦,1388.0,None,False,"[{'id': 3399769248862229720, 'resource_state':...","[{'distance': 1003.5, 'elapsed_time': 137, 'el...","[{'distance': 1614.2, 'elapsed_time': 235, 'el...","[{'id': 56016268643, 'resource_state': 2, 'nam...","[{'type': 'heart_rate', 'visibility': 'everyon...",False,Garmin Edge 840,246e2c4b121e976191c2c4d98a055e939da834c6,,"[heartrate, power]",81055898,1,a15729456618,}a}vH{kogBILs@x@WP_@PGJKp@O|BWxAM`@c@|@o@~@KZ?...,3,kv}vHw~mgBqMlTwFbHa@pA`LzVaAzNy@`Pf@bZFtYrAfPC...,b12572672,False,Cube Nuroad Pro,Cube Nuroad Pro,2,False,3734608,3734.6,None,0


In [40]:
session = requests.Session()
session.headers.update({"Authorization": f"Bearer {access_token}"})
resp = fetch_activity_details(14801524762)
df2 = pd.json_normalize(resp)

In [41]:
df2

,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,id,start_date,start_date_local,timezone,utc_offset,location_city,location_state,location_country,achievement_count,kudos_count,comment_count,athlete_count,photo_count,trainer,commute,manual,private,visibility,flagged,gear_id,start_latlng,end_latlng,average_speed,max_speed,average_cadence,average_watts,max_watts,weighted_average_watts,device_watts,kilojoules,has_heartrate,average_heartrate,max_heartrate,heartrate_opt_out,display_hide_heartrate_option,elev_high,elev_low,upload_id,upload_id_str,external_id,from_accepted_tag,pr_count,total_photo_count,has_kudoed,suffer_score,description,calories,perceived_exertion,prefer_perceived_exertion,segment_efforts,splits_metric,splits_standard,laps,best_efforts,stats_visibility,hide_from_home,device_name,embed_token,available_zones,athlete.id,athlete.resource_state,map.id,map.polyline,map.resource_state,map.summary_polyline,gear.id,gear.primary,gear.name,gear.nickname,gear.resource_state,gear.retired,gear.distance,gear.converted_distance,photos.primary.unique_id,photos.primary.urls.600,photos.primary.urls.100,photos.primary.source,photos.primary.media_type,photos.use_primary_photo,photos.count,similar_activities.effort_count,similar_activities.average_speed,similar_activities.min_average_speed,similar_activities.mid_average_speed,similar_activities.max_average_speed,similar_activities.pr_rank,similar_activities.frequency_milestone,similar_activities.trend.speeds,similar_activities.trend.current_activity_index,similar_activities.trend.min_speed,similar_activities.trend.mid_speed,similar_activities.trend.max_speed,similar_activities.trend.direction,similar_activities.resource_state
0,3,11. PKO Nocny Wrocław Półmaraton🌝,21322.3,6470,6470,38.0,Run,Run,1,14801524762,2025-06-14T20:07:42Z,2025-06-14T22:07:42Z,(GMT+01:00) Europe/Warsaw,7200.0,None,None,None,7,20,5,234,0,False,False,False,False,everyone,False,g23642256,"[51.119096, 17.096763]","[51.119122, 17.096723]",3.296,5.12,85.4,394.7,551,396,True,2553.6,True,168.8,177.0,False,True,125.8,117.4,15796441135,15796441135,garmin_ping_449180416971,False,7,2,False,245.0,Za mocny początek i zabrakło na końcu🫠\nNetto:...,1618.0,None,False,"[{'id': 3369120328444249534, 'resource_state':...","[{'distance': 1003.5, 'elapsed_time': 329, 'el...","[{'distance': 1612.3, 'elapsed_time': 504, 'el...","[{'id': 52640234336, 'resource_state': 2, 'nam...","[{'id': 62249273807, 'resource_state': 2, 'nam...","[{'type': 'heart_rate', 'visibility': 'everyon...",False,Garmin Forerunner 970,1f3a114312a9875121fa0cba838c0b9fa27aed90,"[heartrate, pace, power]",81055898,1,a14801524762,ie_wHwejgBdCQV@NL`@r@FTDTPbJDtADhELxFFxBFnDAVU...,3,ie_wHwejgB|CONL`@r@Lj@z@fd@AVUf@AT\z@Lp@|@nLNx...,g23642256,False,Adidas EVO SL,None,2,False,146967,147.0,51429AA6-F57F-4BDA-9345-8A84288F28CD,https://dgtzuqphqg23d.cloudfront.net/sjp_5Aj9O...,https://dgtzuqphqg23d.cloudfront.net/sjp_5Aj9O...,1,1,False,2,3,3.004799,2.696279,2.965952,3.295564,1,3,"[2.696279011099899, 2.878180754622855, 3.00504...",2,2.696279,2.965952,3.295564,1,2
